In [172]:
#21/09/2022
#Answer Request And Evaluation lv1
#continue with auto_metric by Xiaoya
#update 1: add encoding = "utf-8" when reading the file
#update 2: add if to judge the existence of bgk and postdlg

In [163]:
import requests
import json
import os

import sys
import time

import numpy as np

from evaluate import load

import os.path

import xlsxwriter

from evaluate import load

In [164]:
# create a new Excel file .demo.xlsx and add two worksheets data and metric
workbook = xlsxwriter.Workbook('.demo.xlsx')
worksheet = workbook.add_worksheet('data')
worksheet2 = workbook.add_worksheet('metric')

In [165]:
def test_model(url,dataset_name):
    if dataset_name=="KdConv":
        #the content of KdConv dataset
        path="./dev_XDAI.json"
        #the results
        filename="./dev_result.json" 
    
    #read the dataset
    with open(path, "r", encoding = "utf-8") as f:
        row_data = json.load(f)
    len_row_data=len(row_data)
    
    #fill the worksheet one
    #wirte(row, col, *args)
    worksheet.write(0, 0, "payload")
    worksheet.write(0, 1, "query")
    worksheet.write(0, 2, "reply")
    worksheet.write(0, 3, "process_time")
    #Widen the first column to make the text clearer
    #worksheet.set_column('AA', 20)
    
    #Add a bold format to use to highlight cells
    #在工作表中创建一个新的格式对象来格式化单元格，实现加粗
    bold = workbook.add_format({"bold": True})
    
    count=0
    Results=[]
    input_texts=[]
    
    for d in row_data:
        ##only process five data in the dataset
        #if count==5:
        #    break
        
        #get the necessary content in the data
        query = d["Components"]["query"]["text"]
        dlg_0 = d["Components"]["dlg"][0]["text"]
        dlg_1 = d["Components"]["dlg"][1]["text"]
        qak_0 = d["Components"]["qak"][0]["text"]
        qak_1 = d["Components"]["qak"][1]["text"]
        bgk = ""
        #if background exists
        if "bgk" in d["Components"].keys():
            bgk = d["Components"]["bgk"]
        postdlg = ""
        #if subsequent sentence exists
        if [1, 0][d["Components"]["postdlg"] == []]:
            postdlg = d["Components"]["postdlg"][0]["text"]

        #format the necessary content to fit the interface
        #payload - for interface
        payload = json.dumps({
        "query": query,
        "username": "A",
        "botname": "B",
        "history": [
            {
            "talker": "user",
            "text": str(dlg_0)
            },
            {
            "talker": "bot",
            "text": str(dlg_1)
            }
        ],
        "qapairs": [
            {
            "q": qak_0,
            "a": qak_1
            }
        ],
        "description": bgk,
        "postfix": postdlg,
        "agent": "XDAminer"
        })
        
        #headers - for interface
        headers = {
            'Authorization': 'U8QAAHsp3SuTL1GJmuGKS6UM-8Mt1bdVlkc134w0jio',
            'Content-Type': 'application/json'
        }

        #get response from interface
        response = requests.request("POST", url, headers=headers, data=payload)
        
        #transform str to dict
        s=json.loads(response.text)
        
        #collect the response
        Results.append(s)
        worksheet.write(count+1, 0, str(json.loads(payload)))
        worksheet.write(count+1, 1, str(s["data"]["query"]))
        worksheet.write(count+1, 2, str(s["data"]["reply"]))
        worksheet.write(count+1, 3, str(s["data"]["process_time"]))

        # input_texts.append(s[data][reply])

        count+=1
        print('r',end='')
        print(f'模型测试中,生成reply{count} of {len_row_data}',end='',flush=True)
        #time.sleep(0.001)
    

    #save the results
    with open(filename, 'w', encoding = 'utf-8') as json_file:
        #transform dict to str
        json_str = json.dumps(Results,indent=4,ensure_ascii=False)
        json_file.write(json_str)

    # return input_texts

In [166]:
def load_data(dataset_name):
    #path of dataset and results for KdConv dataset
    if dataset_name=="KdConv":
        path="./dev_XDAI.json"
        filename="./dev_result.json" 
    
    #read the dataset
    with open(path, "r", encoding = "utf-8") as f:
        #read json
        #dataset are in row_data
        row_data = json.load(f)
    #number of data in the dataset
    len_row_data=len(row_data)

    #read the results
    with open(filename, "r", encoding = "utf-8") as f1:
        #transform str to dict
        #results are in contents
        contents = json.loads(f1.read())
        f1.close()
     
    #list for standard answers (list of strings)
    ref = []
    #list for standard answers (list of lists)
    ref_list = []
    #list for generated answers (list of strings)
    gen = []
    #list for generatred answers (list of lists)
    gen_list=[]
        
    cnt=0
    #collect srandard answers in dataset
    for d in row_data:
        #if cnt==5:
        #    break
        ref.append(d["Components"]["qak"][1]["text"])
        ref_list.append(list(d["Components"]["qak"][1]["text"]))
        cnt+=1
    
    count = 0
    #collect generated answers in results
    for content in contents:
        #list of strings
        gen.append(content["data"]["reply"])
        #list of lists
        gen_list.append(list(content["data"]["reply"]))
        count += 1
    
    # print(example , gen[0], gen_list[0],ref_list[0])
    print(f"load {count} gens, {cnt} refs")
    return gen, ref, gen_list, ref_list

In [167]:
def cal_ppl(gen):
    perplexity = load("perplexity", module_type = "metric")#提出来？
    results = perplexity.compute(model_id = 'gpt2',
                                predictions = gen)
    #列表中文本输入的困惑度分数和平均困惑度。 如果输入文本之一长于模型的最大输入长度，则将其截断为最大长度以进行困惑度计算。 
    return results

#Hugging Face - perplexity
#https://huggingface.co/spaces/evaluate-metric/perplexity
#Output example:
#{'perplexities': [8.182524681091309, 33.42122268676758, 27.012239456176758], 'mean_perplexity': 22.871995608011883}

In [168]:
def cal_bleu(mode, gen, ref):
    score = []
    for i in range(0, len(gen)):
        gen_list = gen[i]
        ref_list = ref[i]
        if len(ref_list) < mode:
            continue
        if len(gen_list) < mode:
            score.append(0)
            continue
        ref_block_list = []
        gen_block_list = []
        for k in range(0, len(gen_list) - mode + 1):
            if (k + mode) <= len(gen_list):
                gen_block_list.append("".join(gen_list[k:k + mode]))
        for k in range(0, len(ref_list) - mode + 1):
            if (k + mode) <= len(ref_list):
                ref_block_list.append("".join(ref_list[k:k + mode]))
        # print(ref_block_list)
        # print(gen_block_list)
        overlap = 0.0
        for item in gen_block_list:
            if item in ref_block_list:
                overlap += 1
        score.append(overlap/len(gen_block_list))
    #     print(overlaplen(gen_block_list))
    # print(len(score), scores valid ================)
    score = np.array(score)
    return np.mean(score)

In [169]:
def cal_distinct(mode, gen):
    total_count = 0
    unique_dict = {}
    for i in range(0, len(gen)):
        gen_list = gen[i]
        if len(gen_list) < mode:
            continue
        for k in range(0, len(gen_list) - mode + 1):
            if (k + mode) <= len(gen_list):
                n_gram = "".join(gen_list[k:k + mode])
                total_count += 1
                if n_gram in unique_dict:
                    unique_dict[n_gram] += 1 
                else:
                    unique_dict[n_gram] = 1
    # print(total num , total_count)
    return len(unique_dict) / total_count

In [170]:
def cal_bert(ref, gen):
    #load bertscore
    bertscore = load("bertscore")
    
    bertscore_results = []
    
    #calculate bertscore
    predictions = gen
    references = ref
    result = bertscore.compute(predictions=predictions, references=references, lang="zh")
       
    return result

#Hugging Face - bert_score
#https://huggingface.co/spaces/evaluate-metric/bertscore
#Output example:
#{'precision': [0.7380737066268921, 0.5584042072296143], 'recall': [0.7380737066268921, 0.5889028906822205], 'f1': [0.7380737066268921, 0.5732481479644775], 'hashcode': 'bert-base-uncased_L5_no-idf_version=0.3.10(hug_trans=4.10.3)'}

In [171]:
if __name__=="__main__":

    url="http://aigc.aminer.cn/test/xd/v2/query_plain_qa"
    dataset_name="KdConv"  
    print(f"model: {url}\ndataset: {dataset_name}")

    if dataset_name=="KdConv":
        filename="./dev_result.json"
        path="./dev_XDAI.json"
        
    #test_model(url,dataset_name)

    if os.path.isfile(filename):
        print("已有模型测试结果")
    else:
        test_model(url,dataset_name)
    
    gen, ref, gen_list, ref_list=load_data(dataset_name)

    print("开始评测指标")

    #fill the evaluated results in worksheet 2
    worksheet2.write(0,0,'PPL')
    worksheet2.write(1,0,'BLEU-1234')
    worksheet2.write(2,0,'Distinct-1234')
    worksheet2.write(3,0,'BERTscore-precisionrecallf1')
    
    #PPL
    #判断是否有文件？没有就生成
    results=cal_ppl(gen)
    print("PPL:", round(results["mean_perplexity"], 2))#平均困惑度
    worksheet2.write(0, 1, round(results["mean_perplexity"], 2))

    #BLEU - 1/2/3/4
    print("BLEU-1/2/3/4:", round(cal_bleu(1, gen_list, ref_list)*100, 2), round(cal_bleu(2, gen_list, ref_list)*100, 2), round(cal_bleu(3, gen_list, ref_list)*100,2), round(cal_bleu(4, gen_list, ref_list)*100,2))
    worksheet2.write(1, 1, round(cal_bleu(1, gen_list, ref_list)*100, 2))
    worksheet2.write(1, 2, round(cal_bleu(2, gen_list, ref_list)*100, 2))
    worksheet2.write(1, 3, round(cal_bleu(3, gen_list, ref_list)*100, 2))
    worksheet2.write(1, 4, round(cal_bleu(4, gen_list, ref_list)*100, 2))

    #Distinct - 1/2/3/4
    print("Distinct-1/2/3/4:",round(cal_distinct(1, gen_list)*100,2), round(cal_distinct(2, gen_list)*100,2), round(cal_distinct(3, gen_list)*100,2), round(cal_distinct(4, gen_list)*100,2))
    worksheet2.write(2,1,round(cal_distinct(1, gen_list)*100,2))
    worksheet2.write(2,2,round(cal_distinct(2, gen_list)*100,2))
    worksheet2.write(2,3,round(cal_distinct(3, gen_list)*100,2))
    worksheet2.write(2,4,round(cal_distinct(4, gen_list)*100,2))

    #BERTscore - precision, recall, f1
    bertscore_result=cal_bert(ref, gen)

    precision=sum(bertscore_result["precision"])/len(bertscore_result["precision"])
    recall=sum(bertscore_result["recall"])/len(bertscore_result["recall"])
    f1=sum(bertscore_result["f1"])/len(bertscore_result["f1"])
    print("BERTscore - precision, recall, f1", round(precision*100,2), round(recall*100,2), round(f1*100,2))
    worksheet2.write(3,1,round(precision*100,2))
    worksheet2.write(3,2,round(recall*100,2))
    worksheet2.write(3,3,round(f1*100,2))

    workbook.close()

model: http://aigc.aminer.cn/test/xd/v2/query_plain_qa
dataset: KdConv


r模型测试中,生成reply1 of 17244r模型测试中,生成reply2 of 17244r模型测试中,生成reply3 of 17244r模型测试中,生成reply4 of 17244r模型测试中,生成reply5 of 17244r模型测试中,生成reply6 of 17244r模型测试中,生成reply7 of 17244r模型测试中,生成reply8 of 17244r模型测试中,生成reply9 of 17244r模型测试中,生成reply10 of 17244r模型测试中,生成reply11 of 17244r模型测试中,生成reply12 of 17244r模型测试中,生成reply13 of 17244r模型测试中,生成reply14 of 17244r模型测试中,生成reply15 of 17244r模型测试中,生成reply16 of 17244r模型测试中,生成reply17 of 17244r模型测试中,生成reply18 of 17244r模型测试中,生成reply19 of 17244r模型测试中,生成reply20 of 17244r模型测试中,生成reply21 of 17244r模型测试中,生成reply22 of 17244r模型测试中,生成reply23 of 17244r模型测试中,生成reply24 of 17244r模型测试中,生成reply25 of 17244r模型测试中,生成reply26 of 17244r模型测试中,生成reply27 of 17244r模型测试中,生成reply28 of 17244r模型测试中,生成reply29 of 17244r模型测试中,生成reply30 of 17244r模型测试中,生成reply31 of 17244r模型测试中,生成reply32 of 17244r模型测试中,生成reply33 of 17244r模型测试中,生成reply34 of 17244r模型测试中,生成reply35 of 17244r模型测试中,生成reply36 of 17244r模型测试中,生成reply37 of 17244r模型测试中,生成reply38 of 17244r模型测试中,生成reply39 of 17244r模型测试中,生成reply40 of 17244r模型测试中,生成

r模型测试中,生成reply321 of 17244r模型测试中,生成reply322 of 17244r模型测试中,生成reply323 of 17244r模型测试中,生成reply324 of 17244r模型测试中,生成reply325 of 17244r模型测试中,生成reply326 of 17244r模型测试中,生成reply327 of 17244r模型测试中,生成reply328 of 17244r模型测试中,生成reply329 of 17244r模型测试中,生成reply330 of 17244r模型测试中,生成reply331 of 17244r模型测试中,生成reply332 of 17244r模型测试中,生成reply333 of 17244r模型测试中,生成reply334 of 17244r模型测试中,生成reply335 of 17244r模型测试中,生成reply336 of 17244r模型测试中,生成reply337 of 17244r模型测试中,生成reply338 of 17244r模型测试中,生成reply339 of 17244r模型测试中,生成reply340 of 17244r模型测试中,生成reply341 of 17244r模型测试中,生成reply342 of 17244r模型测试中,生成reply343 of 17244r模型测试中,生成reply344 of 17244r模型测试中,生成reply345 of 17244r模型测试中,生成reply346 of 17244r模型测试中,生成reply347 of 17244r模型测试中,生成reply348 of 17244r模型测试中,生成reply349 of 17244r模型测试中,生成reply350 of 17244r模型测试中,生成reply351 of 17244r模型测试中,生成reply352 of 17244r模型测试中,生成reply353 of 17244r模型测试中,生成reply354 of 17244r模型测试中,生成reply355 of 17244r模型测试中,生成reply356 of 17244r模型测试中,生成reply357 of 17244r模型测试中,生成reply358 of 17244r模型测试中,生成rep

r模型测试中,生成reply637 of 17244r模型测试中,生成reply638 of 17244r模型测试中,生成reply639 of 17244r模型测试中,生成reply640 of 17244r模型测试中,生成reply641 of 17244r模型测试中,生成reply642 of 17244r模型测试中,生成reply643 of 17244r模型测试中,生成reply644 of 17244r模型测试中,生成reply645 of 17244r模型测试中,生成reply646 of 17244r模型测试中,生成reply647 of 17244r模型测试中,生成reply648 of 17244r模型测试中,生成reply649 of 17244r模型测试中,生成reply650 of 17244r模型测试中,生成reply651 of 17244r模型测试中,生成reply652 of 17244r模型测试中,生成reply653 of 17244r模型测试中,生成reply654 of 17244r模型测试中,生成reply655 of 17244r模型测试中,生成reply656 of 17244r模型测试中,生成reply657 of 17244r模型测试中,生成reply658 of 17244r模型测试中,生成reply659 of 17244r模型测试中,生成reply660 of 17244r模型测试中,生成reply661 of 17244r模型测试中,生成reply662 of 17244r模型测试中,生成reply663 of 17244r模型测试中,生成reply664 of 17244r模型测试中,生成reply665 of 17244r模型测试中,生成reply666 of 17244r模型测试中,生成reply667 of 17244r模型测试中,生成reply668 of 17244r模型测试中,生成reply669 of 17244r模型测试中,生成reply670 of 17244r模型测试中,生成reply671 of 17244r模型测试中,生成reply672 of 17244r模型测试中,生成reply673 of 17244r模型测试中,生成reply674 of 17244r模型测试中,生成rep

ConnectionError: HTTPConnectionPool(host='aigc.aminer.cn', port=80): Max retries exceeded with url: /test/xd/v2/query_plain_qa (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002EC2BD61220>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))